# **mBART fine tuning for English and Sinhala text simplification**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Check GPU

In [ ]:
!nvidia-smi

Sat Nov 20 03:32:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Libraries

In [ ]:
!wget https://github.com/pytorch/fairseq/archive/refs/tags/v0.10.2.tar.gz && \
tar -xf v0.10.2.tar.gz && \
cd fairseq-0.10.2 && \
pip install --editable ./ -q
!pip install fastBPE sacremoses subword_nmt sentencepiece -q

--2021-11-20 03:32:24--  https://github.com/pytorch/fairseq/archive/refs/tags/v0.10.2.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/pytorch/fairseq/tar.gz/refs/tags/v0.10.2 [following]
--2021-11-20 03:32:24--  https://codeload.github.com/pytorch/fairseq/tar.gz/refs/tags/v0.10.2
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘v0.10.2.tar.gz’

v0.10.2.tar.gz          [  <=>               ]   2.94M  14.5MB/s    in 0.2s    

2021-11-20 03:32:24 (14.5 MB/s) - ‘v0.10.2.tar.gz’ saved [3088040]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done

## Import mBart-50 model

In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart50/mbart50.pretrained.tar.gz
!tar -xzvf mbart50.pretrained.tar.gz

--2021-11-20 03:33:54--  https://dl.fbaipublicfiles.com/fairseq/models/mbart50/mbart50.pretrained.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7199950118 (6.7G) [application/gzip]
Saving to: ‘mbart50.pretrained.tar.gz’

mbart50.pretrained. 100%[===================>]   6.71G  21.0MB/s    in 5m 27s  

2021-11-20 03:39:22 (21.0 MB/s) - ‘mbart50.pretrained.tar.gz’ saved [7199950118/7199950118]

mbart50.pretrained/
mbart50.pretrained/sentence.bpe.model
mbart50.pretrained/dict.af_ZA.txt
mbart50.pretrained/dict.ar_AR.txt
mbart50.pretrained/dict.az_AZ.txt
mbart50.pretrained/dict.bn_IN.txt
mbart50.pretrained/dict.cs_CZ.txt
mbart50.pretrained/dict.de_DE.txt
mbart50.pretrained/dict.en_XX.txt
mbart50.pretrained/dict.es_XX.txt
mbart50.pretrained/dict.et_EE.txt
mbart50.pretrained/dict

## **En - Simplification**

### Import simplification data

In [ ]:
!rm -f download
!wget https://dms.uom.lk/s/QfsPWZ3zLZ59bAD/download
!unzip download
!rm -f download

--2021-11-18 06:50:23--  https://dms.uom.lk/s/QfsPWZ3zLZ59bAD/download
Resolving dms.uom.lk (dms.uom.lk)... 192.248.8.71, 2401:dd00:10:1::71
Connecting to dms.uom.lk (dms.uom.lk)|192.248.8.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘download’

download                [              <=>   ]  31.19M  9.20MB/s    in 3.4s    

2021-11-18 06:50:28 (9.20 MB/s) - ‘download’ saved [32702413]

Archive:  download
   creating: newsela-en/
 extracting: newsela-en/train.complex  
 extracting: newsela-en/train.simple  
 extracting: newsela-en/valid.complex  
 extracting: newsela-en/valid.simple  


In [ ]:
!rm -f download
!wget https://dms.uom.lk/s/DmjSWj8Zs3FzrCi/download
!unzip download
!rm -f download

--2021-11-18 06:50:29--  https://dms.uom.lk/s/DmjSWj8Zs3FzrCi/download
Resolving dms.uom.lk (dms.uom.lk)... 192.248.8.71, 2401:dd00:10:1::71
Connecting to dms.uom.lk (dms.uom.lk)|192.248.8.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘download’

download                [    <=>             ] 418.63K   666KB/s    in 0.6s    

2021-11-18 06:50:31 (666 KB/s) - ‘download’ saved [428680]

Archive:  download
   creating: asset/
 extracting: asset/asset.valid.orig  
 extracting: asset/asset.valid.simp.0  


In [ ]:
!mkdir drive/Shareddrives/SiTS/mBart-Finetune/new-experiments drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8 drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/preprocess
!python fairseq-0.10.2/scripts/spm_encode.py \
  --model mbart50.pretrained/sentence.bpe.model \
  --inputs newsela-en/train.simple newsela-en/train.complex \
  --outputs drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/train.bpe.simple drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/train.bpe.complex

!python fairseq-0.10.2/scripts/spm_encode.py \
    --model mbart50.pretrained/sentence.bpe.model \
    --inputs newsela-en/valid.simple newsela-en/valid.complex \
    --outputs drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/valid.bpe.simple drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/valid.bpe.complex

!python fairseq-0.10.2/scripts/spm_encode.py \
    --model mbart50.pretrained/sentence.bpe.model \
    --inputs asset/asset.valid.simp.0 asset/asset.valid.orig \
    --outputs drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/test.bpe.simple drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/test.bpe.complex

DICT_s="mbart50.pretrained/dict.en_XX.txt"

!fairseq-preprocess \
  --source-lang complex \
  --target-lang simple \
  --trainpref drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/train.bpe \
  --validpref drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/valid.bpe \
  --testpref drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/spm/test.bpe \
  --destdir drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/preprocess \
  --srcdict $DICT_s \
  --joined-dictionary \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --workers 70

### Start training

In [ ]:
path_2_data="drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/preprocess"
# pretrained_model="mbart50.pretrained/model.pt"
pretrained_model = "drive/Shareddrives/SiTS/mBart-Finetune/pretrained-translation-model/checkpoints/checkpoint_best.pt"
!fairseq-train $path_2_data \
  --restore-file $pretrained_model \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --langs "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN,af_ZA,az_AZ,bn_IN,fa_IR,he_IL,hr_HR,id_ID,ka_GE,km_KH,mk_MK,ml_IN,mn_MN,mr_IN,pl_PL,ps_AF,pt_XX,sv_SE,sw_KE,ta_IN,te_IN,th_TH,tl_XX,uk_UA,ur_PK,xh_ZA,gl_ES,sl_SI" \
  --source-lang complex --target-lang simple \
  --memory-efficient-fp16 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler inverse_sqrt --lr 3e-05 --warmup-updates 2500 --max-update 20000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --reset-optimizer \
  --save-interval 1 --save-interval-updates 5000 --keep-interval-updates 10 --no-epoch-checkpoints \
  --seed 222 --log-format simple --log-interval 2 \
  --save-dir drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/checkpoints > drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/logs 2>&1 & disown

In [ ]:
!tail -f drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/logs

2021-11-18 13:09:56 | INFO | train_inner | epoch 028:   1224 / 3130 loss=5.717, nll_loss=1.905, ppl=3.74, wps=1054.2, ups=1.12, wpb=938.5, bsz=48, num_updates=19996, lr=1.06077e-05, gnorm=1.807, clip=None, loss_scale=2, train_wall=2, wall=0
2021-11-18 13:09:58 | INFO | train_inner | epoch 028:   1226 / 3130 loss=5.658, nll_loss=1.83, ppl=3.56, wps=1345.5, ups=0.9, wpb=1502.5, bsz=72, num_updates=19998, lr=1.06071e-05, gnorm=1.315, clip=None, loss_scale=2, train_wall=2, wall=0
2021-11-18 13:10:00 | INFO | train_inner | epoch 028:   1228 / 3130 loss=5.877, nll_loss=2.103, ppl=4.3, wps=1436.8, ups=0.83, wpb=1738, bsz=76, num_updates=20000, lr=1.06066e-05, gnorm=1.365, clip=None, loss_scale=2, train_wall=2, wall=0
2021-11-18 13:10:00 | INFO | fairseq_cli.train | begin validation on "valid" subset
2021-11-18 13:10:16 | INFO | valid | epoch 028 | valid on 'valid' subset | loss 5.871 | nll_loss 1.895 | ppl 3.72 | wps 2967.9 | wpb 482.3 | bsz 20.4 | num_updates 20000 | best_loss 5.871
2021-11-

## **Si - Simplification**

### Import simplification data

In [ ]:
!gdown --id 1Vm26F4mAU97K7y41ADpmyjvcwhSRvVfE
!tar xf newsela-splitted-sineval.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1Vm26F4mAU97K7y41ADpmyjvcwhSRvVfE
To: /content/newsela-splitted-sineval.tar.gz
100% 6.75M/6.75M [00:00<00:00, 21.5MB/s]


### Preprocess data

In [ ]:
!mkdir drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/preprocess
!python fairseq-0.10.2/scripts/spm_encode.py \
  --model mbart50.pretrained/sentence.bpe.model \
  --inputs newsela-splitted-sineval/train.simple newsela-splitted-sineval/train.complex \
  --outputs drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/train.bpe.simple drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/train.bpe.complex

!python fairseq-0.10.2/scripts/spm_encode.py \
    --model mbart50.pretrained/sentence.bpe.model \
    --inputs newsela-splitted-sineval/valid.simple newsela-splitted-sineval/valid.complex \
    --outputs drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/valid.bpe.simple drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/valid.bpe.complex

!python fairseq-0.10.2/scripts/spm_encode.py \
    --model mbart50.pretrained/sentence.bpe.model \
    --inputs newsela-splitted-sineval/test.simple newsela-splitted-sineval/test.complex \
    --outputs drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/test.bpe.simple drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/test.bpe.complex

DICT_s="mbart50.pretrained/dict.si_LK.txt"

!fairseq-preprocess \
  --source-lang complex \
  --target-lang simple \
  --trainpref drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/train.bpe \
  --validpref drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/valid.bpe \
  --testpref drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/spm/test.bpe \
  --destdir drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/preprocess \
  --srcdict $DICT_s \
  --joined-dictionary \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --workers 70

### Start training

In [ ]:
path_2_data="drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/preprocess"
pretrained_model="drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/en-sm/checkpoints/checkpoint_last.pt"
last_checkpoint = "drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/checkpoints/checkpoint_last.pt"

!fairseq-train $path_2_data \
  --restore-file $last_checkpoint \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --langs "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN,af_ZA,az_AZ,bn_IN,fa_IR,he_IL,hr_HR,id_ID,ka_GE,km_KH,mk_MK,ml_IN,mn_MN,mr_IN,pl_PL,ps_AF,pt_XX,sv_SE,sw_KE,ta_IN,te_IN,th_TH,tl_XX,uk_UA,ur_PK,xh_ZA,gl_ES,sl_SI" \
  --source-lang complex --target-lang simple \
  --memory-efficient-fp16 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler inverse_sqrt --lr 3e-05 --warmup-updates 2500 --max-update 10000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --reset-optimizer \
  --save-interval 1 --save-interval-updates 2000 --keep-interval-updates 10 --no-epoch-checkpoints \
  --seed 222 --log-format simple --log-interval 2 \
  --save-dir drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/checkpoints > drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/logs 2>&1 & disown

In [ ]:
!tail -f drive/Shareddrives/SiTS/mBart-Finetune/new-experiments/model8/si-sm/logs

2021-11-20 06:59:13 | INFO | train_inner | epoch 034:    904 / 3388 loss=6.154, nll_loss=2.422, ppl=5.36, wps=1410, ups=0.96, wpb=1464.5, bsz=40, num_updates=9996, lr=1.5003e-05, gnorm=2.493, clip=None, loss_scale=16, train_wall=2, wall=0
2021-11-20 06:59:16 | INFO | train_inner | epoch 034:    906 / 3388 loss=5.848, nll_loss=2.063, ppl=4.18, wps=1325.3, ups=0.92, wpb=1437.5, bsz=52, num_updates=9998, lr=1.50015e-05, gnorm=7.782, clip=None, loss_scale=16, train_wall=2, wall=0
2021-11-20 06:59:17 | INFO | train_inner | epoch 034:    908 / 3388 loss=5.95, nll_loss=2.17, ppl=4.5, wps=986.3, ups=1.16, wpb=847.5, bsz=28, num_updates=10000, lr=1.5e-05, gnorm=1.873, clip=None, loss_scale=16, train_wall=2, wall=0
2021-11-20 06:59:17 | INFO | fairseq_cli.train | begin validation on "valid" subset
2021-11-20 06:59:36 | INFO | valid | epoch 034 | valid on 'valid' subset | loss 6.264 | nll_loss 2.27 | ppl 4.82 | wps 3130.9 | wpb 498.6 | bsz 17.2 | num_updates 10000 | best_loss 6.256
2021-11-20 06:

## Session Run

In [ ]:
!touch test.log

In [ ]:
!tail -f test.log

^C


In [ ]:
!top

=top - 13:05:56 up  2:18,  0 users,  load average: 0.92, 1.06, 1.26
Tasks:  16 total,   1 running,  15 sleeping,   0 stopped,   0 zombie
%Cpu(s): 13.3 us,  7.8 sy,  0.0 ni, 72.4 id,  6.1 wa,  0.0 hi,  0.4 si,  0.0 st
KiB Mem : 26696424 total, 11627500 free,   770296 used, 14298628 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 25512112 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND   
     88 root      20   0  955464  89476   4976 S   6.7  0.3   0:35.77 python3   
      1 root      20   0     992      4      0 S   0.0  0.0   0:00.54 docker-i+ 
      7 root      20   0  346728  34184   9328 S   0.0  0.1   0:05.17 node      
     17 root      20   0   35888   1188     96 S   0.0  0.0   0:00.82 tail      
     51 root      20   0  160352  29256   1112 S   0.0  0.1   0:00.29 colab-fi+ 
     76 root      20   0  194928  49504   3036 S   0.0  0.2   0:06.70 jupyter-+ 
     77 root      20   0  708340   7888   2364 S   0.0  0.0   

In [ ]:
!kill 1991